In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None)
from helper_functions import separate_continous_discrete, print_correlation_matrix, plot_discrete_variables, plot_continuous_variables, plot_outliers, remove_outlier_iqr, remove_outlier_cutoff_between

**Instructions**
1. Open the notebook created for Lab-cleaning-numerical-data.
2. Find all of the categorical data. Save it in a categorical_df variable.

In [2]:
customer_df = pd.read_csv('customer.csv')
customer_df.rename(columns=lambda x: x.strip().replace(" ", "_").lower(), inplace=True)
categorical_df = customer_df.select_dtypes(include='object')
display(categorical_df.shape)
categorical_df.head()

(8290, 16)

,customer,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,No,Basic,Bachelor,2011-02-24,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,2011-01-20,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,2011-02-03,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize


3. Check for NaN values and decide what to do with them, do it now.

In [3]:
categorical_df.isna().sum().sum()
# no nan values

0

4. Check all unique values of columns.

In [4]:
categorical_df.apply(lambda x: x.nunique())

customer             8290
state                   5
response                2
coverage                3
education               5
effective_to_date      59
employmentstatus        5
gender                  2
location_code           3
marital_status          3
policy_type             3
policy                  9
renew_offer_type        4
sales_channel           4
vehicle_class           6
vehicle_size            3
dtype: int64

In [5]:
{column: categorical_df[column].unique() for column in categorical_df.columns}

{'customer': array(['BU79786', 'QZ44356', 'AI49188', ..., 'TD14365', 'UP19263',
        'Y167826'], dtype=object),
 'state': array(['Washington', 'Arizona', 'Nevada', 'California', 'Oregon'],
       dtype=object),
 'response': array(['No', 'Yes'], dtype=object),
 'coverage': array(['Basic', 'Extended', 'Premium'], dtype=object),
 'education': array(['Bachelor', 'College', 'Master', 'High School or Below', 'Doctor'],
       dtype=object),
 'effective_to_date': array(['2011-02-24', '2011-01-31', '2011-02-19', '2011-01-20',
        '2011-02-03', '2011-01-25', '2011-01-18', '2011-02-17',
        '2011-02-21', '2011-01-06', '2011-02-06', '2011-01-10',
        '2011-01-17', '2011-01-05', '2011-02-27', '2011-01-14',
        '2011-01-21', '2011-02-05', '2011-01-29', '2011-02-28',
        '2011-02-12', '2011-02-02', '2011-02-07', '2011-02-13',
        '2011-01-15', '2011-01-08', '2011-01-11', '2011-01-28',
        '2011-02-08', '2011-02-23', '2011-01-02', '2011-02-16',
        '2011-01-22', '20

5. Check dtypes. Do they all make sense as categorical data?

In [6]:
# effective_to_date should be of type date
categorical_df['effective_to_date'] = pd.to_datetime(categorical_df['effective_to_date'])
# check 
categorical_df.dtypes

customer                     object
state                        object
response                     object
coverage                     object
education                    object
effective_to_date    datetime64[ns]
employmentstatus             object
gender                       object
location_code                object
marital_status               object
policy_type                  object
policy                       object
renew_offer_type             object
sales_channel                object
vehicle_class                object
vehicle_size                 object
dtype: object

6. Does any column contain alpha and numeric data? Decide how to clean it and do it now.

In [7]:
# column renew_offer_type can be 1-4
categorical_df['renew_offer_type'] = categorical_df['renew_offer_type'].str.extract('(\d)')
categorical_df['renew_offer_type'].unique()

array(['1', '3', '2', '4'], dtype=object)

7. Would you choose to do anything else to clean or wrangle the categorical data? Comment your decisions and do it now.

In [8]:
# vehicle_size has one option medsize-- should be midsize 
categorical_df['vehicle_size'] = categorical_df['vehicle_size'].replace('Medsize', 'Midsize')
categorical_df['vehicle_size'].unique()

array(['Midsize', 'Small', 'Large'], dtype=object)

In [9]:
# pretty much encoded, change to binary 
categorical_df['response'] = categorical_df['response'].replace({'Yes': 1, 'No': 0})
categorical_df['response'].unique()

array([0, 1])

In [10]:
# policy, repeated type of Personal/Corporate, redundant 
categorical_df['policy'] = categorical_df['policy'].str.extract('([A-Z](\d+))')[1]
categorical_df['policy'].unique()

array(['3', '2', '1'], dtype=object)

In [11]:
# policy_type, repeated mention of Auto, redundant 
categorical_df['policy_type'] = categorical_df['policy_type'].str.extract('^(\w+)')
categorical_df['policy_type'].unique()

array(['Corporate', 'Personal', 'Special'], dtype=object)

In [12]:
categorical_df.head()

,customer,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,0,Basic,Bachelor,2011-02-24,Employed,F,Suburban,Married,Corporate,3,1,Agent,Two-Door Car,Midsize
1,QZ44356,Arizona,0,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal,3,3,Agent,Four-Door Car,Midsize
2,AI49188,Nevada,0,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal,3,1,Agent,Two-Door Car,Midsize
3,WW63253,California,0,Basic,Bachelor,2011-01-20,Unemployed,M,Suburban,Married,Corporate,2,1,Call Center,SUV,Midsize
4,HB64268,Washington,0,Basic,Bachelor,2011-02-03,Employed,M,Rural,Single,Personal,1,1,Agent,Four-Door Car,Midsize


8. Compare policy_type and policy. What information is contained in these columns. Can you identify what is important?
> already did that in previous part

9. Check number of unique values in each column, can they be combined in any way to ease encoding? Comment your thoughts and make those changes.

In [13]:
categorical_df.apply(lambda x: x.nunique())

customer             8290
state                   5
response                2
coverage                3
education               5
effective_to_date      59
employmentstatus        5
gender                  2
location_code           3
marital_status          3
policy_type             3
policy                  3
renew_offer_type        4
sales_channel           4
vehicle_class           6
vehicle_size            3
dtype: int64

In [14]:
{column: categorical_df[column].unique() for column in categorical_df.columns}

{'customer': array(['BU79786', 'QZ44356', 'AI49188', ..., 'TD14365', 'UP19263',
        'Y167826'], dtype=object),
 'state': array(['Washington', 'Arizona', 'Nevada', 'California', 'Oregon'],
       dtype=object),
 'response': array([0, 1]),
 'coverage': array(['Basic', 'Extended', 'Premium'], dtype=object),
 'education': array(['Bachelor', 'College', 'Master', 'High School or Below', 'Doctor'],
       dtype=object),
 'effective_to_date': <DatetimeArray>
 ['2011-02-24 00:00:00', '2011-01-31 00:00:00', '2011-02-19 00:00:00',
  '2011-01-20 00:00:00', '2011-02-03 00:00:00', '2011-01-25 00:00:00',
  '2011-01-18 00:00:00', '2011-02-17 00:00:00', '2011-02-21 00:00:00',
  '2011-01-06 00:00:00', '2011-02-06 00:00:00', '2011-01-10 00:00:00',
  '2011-01-17 00:00:00', '2011-01-05 00:00:00', '2011-02-27 00:00:00',
  '2011-01-14 00:00:00', '2011-01-21 00:00:00', '2011-02-05 00:00:00',
  '2011-01-29 00:00:00', '2011-02-28 00:00:00', '2011-02-12 00:00:00',
  '2011-02-02 00:00:00', '2011-02-07 00:00:0

In [15]:
# effective_to_date, all 2011, jan and feb, can be combined by months 
categorical_df['effective_to_month_in_2011'] = categorical_df['effective_to_date'].dt.month
#categorical_df.drop(columns='effective_to_date')

**leaving the rest as is**

10. Save the cleaned catagorical dataframe as categorical.csv You will use this file again this week.

In [16]:
categorical_df.to_csv('categorical.csv', index=False)
customer_df.to_csv('customer.csv', index=False)